In [1]:
# !pip install tensorflow-gpu  # stable

In [19]:
import numpy as np
import tensorflow as tf

! killall tensorboard
import os
os.system("tensorboard --logdir=./logs --port=7007 &");

In [3]:
tf.reset_default_graph()
x = tf.get_variable("x",shape=(),dtype=tf.float32)
f = x**2
f = tf.Print(f, [x, f], "x, f:")

tf.summary.scalar('curr_x', x)
tf.summary.scalar('curr_f', f)
summaries = tf.summary.merge_all()

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.print instead of tf.Print. Note that tf.print returns a no-output operator that directly prints the output. Outside of defuns or eager mode, this operator will not be executed unless it is directly specified in session.run or used as a control dependency for other operators. This is only a concern in graph mode. Below is an example of how to ensure tf.print executes in graph mode:
```python
    sess = tf.Session()
    with sess.as_default():
        tensor = tf.range(10)
        print_op = tf.print(tensor)
        with tf.control_dependencies([print_op]):
          out = tf.add(tensor, tensor)
        sess.run(out)
    ```
Additionally, to use tf.print in python 2.7, users must make sure to import
the following:

  `from __future__ import print_function`



In [4]:
optimizer = tf.train.GradientDescentOptimizer(0.1)
step = optimizer.minimize(f)

In [5]:
tf.trainable_variables()

[<tf.Variable 'x:0' shape=() dtype=float32_ref>]

In [6]:
s = tf.InteractiveSession()
summary_writer = tf.summary.FileWriter("logs/" + str(np.random.randint(1,21)*5), s.graph)
s.run(tf.global_variables_initializer())

In [7]:
for i in range(10):
    _, curr_summaries = s.run([step, summaries])
    summary_writer.add_summary(curr_summaries, i)
    summary_writer.flush()

In [8]:
tf.InteractiveSession.close(s)

# example with random data

In [14]:
N = 1000
D = 3
x = np.random.random((N,D))
w = np.random.random((D,1))
y = x@w + np.random.randn(N,1)*0.20

In [15]:
tf.reset_default_graph()
features = tf.placeholder(tf.float32, shape=(None,D))
target = tf.placeholder(tf.float32, shape=(None,1))

weights = tf.get_variable("w", shape=(D,1), dtype=tf.float32)
predictions = features@weights

loss = tf.reduce_mean((target - predictions)**2)

optimizer = tf.train.GradientDescentOptimizer(0.1)
step = optimizer.minimize(loss)

tf.summary.scalar('loss', loss)
tf.summary.scalar('weights', tf.reduce_mean(weights))
summaries = tf.summary.merge_all()

In [16]:
s = tf.InteractiveSession()
# saver = tf.train.Saver(tf.trainable_variables())
summary_writer = tf.summary.FileWriter("logs/" + str(np.random.randint(1,21)*5), s.graph)
s.run(tf.global_variables_initializer())

for i in range(300):
#     _, curr_loss, curr_weights = s.run([step, loss, weights], feed_dict={features:x, target:y})
    _, curr_summaries = s.run([step, summaries], feed_dict={features:x, target:y})
    summary_writer.add_summary(curr_summaries, i)
    summary_writer.flush()
    
#     if i % 50 == 0:
#         saver.save(s, "logs/2/model.ckpt", global_step=i)
#         print(curr_loss)
        

In [18]:
tf.InteractiveSession.close(s)